# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [126]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

In [127]:
import homework2

### Install and Load Required Libraries  

In [128]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [129]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [130]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [131]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [132]:
# Some helper functions. These are the same as what the autograder runs.

In [133]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [134]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [135]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [136]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)
        
        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)
        
        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)
        
        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [137]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()
        
        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        
        return acc
    
    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()
            
            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))
            
            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [138]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [139]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 10, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [140]:
# 1. Paths, labels, waveforms

In [141]:
# Once you've written the corresponding code in homework2.py, print these out or visualize them if you want
homework2.waveforms
homework2.labels

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,


In [142]:
# 2. MFCC

In [143]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mfcc,
     homework2.MLPClassifier(),
     0.0001,
     "best_mlp_model.weights")

Extracting features...
Epoch 0, loss = 13.097399248017204, validation accuracy = 0.44715447154471544
Epoch 1, loss = 4.679355962408914, validation accuracy = 0.3008130081300813
Epoch 2, loss = 0.6740733198821545, validation accuracy = 0.7642276422764228
Epoch 3, loss = 0.3774567064311769, validation accuracy = 0.8699186991869918
Epoch 4, loss = 0.2783403789831532, validation accuracy = 0.9105691056910569
Epoch 5, loss = 0.22201211812595525, validation accuracy = 0.9512195121951219
Epoch 6, loss = 0.18384593849380812, validation accuracy = 0.959349593495935
Epoch 7, loss = 0.15421810746192932, validation accuracy = 0.959349593495935
Epoch 8, loss = 0.12939135616438258, validation accuracy = 0.967479674796748
Epoch 9, loss = 0.10887817417581876, validation accuracy = 0.983739837398374
Final validation accuracy = 0.983739837398374, best = 0.983739837398374
Test accuracy = 0.9758064516129032


In [144]:
# 3. Spectrogram

In [145]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.0001,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 0.6326413849989573, validation accuracy = 0.9024390243902439
Epoch 1, loss = 0.5699292139874564, validation accuracy = 0.9186991869918699
Epoch 2, loss = 0.5416372567415237, validation accuracy = 0.9186991869918699
Epoch 3, loss = 0.5191576878229777, validation accuracy = 0.9186991869918699
Epoch 4, loss = 0.49408695929580265, validation accuracy = 0.9186991869918699
Epoch 5, loss = 0.46558961603376603, validation accuracy = 0.926829268292683
Epoch 6, loss = 0.4361928751071294, validation accuracy = 0.9186991869918699
Epoch 7, loss = 0.4084683714641465, validation accuracy = 0.9349593495934959
Epoch 8, loss = 0.3832208514213562, validation accuracy = 0.959349593495935
Epoch 9, loss = 0.36002978930870694, validation accuracy = 0.959349593495935
Final validation accuracy = 0.959349593495935, best = 0.959349593495935
Test accuracy = 0.8387096774193549


In [146]:
# 4. Mel-spectrogram

In [147]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mel,
     homework2.SimpleCNN(),
     0.0001,
     "best_mel_model.weights")

Extracting features...
Epoch 0, loss = 0.4973375110162629, validation accuracy = 0.8211382113821138
Epoch 1, loss = 0.3461119193169806, validation accuracy = 0.8861788617886179
Epoch 2, loss = 0.2898193279074298, validation accuracy = 0.926829268292683
Epoch 3, loss = 0.2535475794639852, validation accuracy = 0.959349593495935
Epoch 4, loss = 0.22609419520530435, validation accuracy = 0.975609756097561
Epoch 5, loss = 0.20433718483481142, validation accuracy = 0.991869918699187
Epoch 6, loss = 0.18600475953684914, validation accuracy = 1.0
Epoch 7, loss = 0.16972224869661862, validation accuracy = 1.0
Epoch 8, loss = 0.15508751571178436, validation accuracy = 1.0
Epoch 9, loss = 0.14196883286866877, validation accuracy = 1.0
Final validation accuracy = 1.0, best = 1.0
Test accuracy = 0.9596774193548387


In [148]:
# 5. Constant-Q

In [149]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_q_model.weights")

Extracting features...
Epoch 0, loss = 0.5441773707667986, validation accuracy = 0.8455284552845529
Epoch 1, loss = 0.44651736153496635, validation accuracy = 0.8699186991869918
Epoch 2, loss = 0.3947448904315631, validation accuracy = 0.9024390243902439
Epoch 3, loss = 0.3575179792112774, validation accuracy = 0.9349593495934959
Epoch 4, loss = 0.3271418511867523, validation accuracy = 0.967479674796748
Epoch 5, loss = 0.2994896773662832, validation accuracy = 0.983739837398374
Epoch 6, loss = 0.27377381838030285, validation accuracy = 0.983739837398374
Epoch 7, loss = 0.24958333124717078, validation accuracy = 0.983739837398374
Epoch 8, loss = 0.22703295449415842, validation accuracy = 0.991869918699187
Epoch 9, loss = 0.20617524161934853, validation accuracy = 0.991869918699187
Final validation accuracy = 0.991869918699187, best = 0.991869918699187
Test accuracy = 0.9919354838709677


In [150]:
# 6. Pitch shift

In [151]:
test(homework2.augmented_waveforms,
     homework2.augmented_labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_augmented_model.weights")

Extracting features...
Epoch 0, loss = 0.5062554113014981, validation accuracy = 0.8997289972899729
Epoch 1, loss = 0.3935260752009021, validation accuracy = 0.959349593495935
Epoch 2, loss = 0.33846193041514466, validation accuracy = 0.9728997289972899
Epoch 3, loss = 0.30133274307957403, validation accuracy = 0.975609756097561
Epoch 4, loss = 0.2710237799695245, validation accuracy = 0.981029810298103
Epoch 5, loss = 0.24433376143376032, validation accuracy = 0.981029810298103
Epoch 6, loss = 0.2204253456244866, validation accuracy = 0.983739837398374
Epoch 7, loss = 0.19997559791361844, validation accuracy = 0.989159891598916
Epoch 8, loss = 0.18275746151253028, validation accuracy = 0.991869918699187
Epoch 9, loss = 0.16818101483362694, validation accuracy = 0.991869918699187
Final validation accuracy = 0.991869918699187, best = 0.991869918699187
Test accuracy = 0.9891891891891892


In [152]:
# 7. Extend your model to handle four classes and creatively improve its performance

In [153]:
test(homework2.waveforms,
     homework2.labels_7,
     homework2.feature_func_7,
     homework2.model_7,
     0.0001,
     "best_model_7.weights")

Extracting features...
Epoch 0, loss = 0.18613026363568175, validation accuracy = 0.8536585365853658
Epoch 1, loss = 0.23602753743115398, validation accuracy = 0.8536585365853658
Epoch 2, loss = 0.2746693326367272, validation accuracy = 0.8455284552845529
Epoch 3, loss = 0.26165690190262264, validation accuracy = 0.8536585365853658
Epoch 4, loss = 0.26695571281015873, validation accuracy = 0.8455284552845529
Epoch 5, loss = 0.33580411598086357, validation accuracy = 0.8536585365853658
Epoch 6, loss = 0.25671067295802963, validation accuracy = 0.8455284552845529
Epoch 7, loss = 0.27157678827643394, validation accuracy = 0.8455284552845529
Epoch 8, loss = 0.32907177983886665, validation accuracy = 0.8455284552845529
Epoch 9, loss = 0.31243132261766327, validation accuracy = 0.8455284552845529
Final validation accuracy = 0.8455284552845529, best = 0.8536585365853658
Test accuracy = 0.8951612903225806
